<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
    <font color='red'>To begin: Click anywhere in this cell and press <kbd>Run</kbd> on the menu bar. This executes the current cell and then highlights the next cell. There are two types of cells. A <i>text cell</i> and a <i>code cell</i>. When you <kbd>Run</kbd> a text cell (<i>we are in a text cell now</i>), you advance to the next cell without executing any code. When you <kbd>Run</kbd> a code cell (<i>identified by <span style="font-family: courier; color:black; background-color:white;">In[ ]:</span> to the left of the cell</i>) you advance to the next cell after executing all the Python code within that cell. Any visual results produced by the code (text/figures) are reported directly below that cell. Press <kbd>Run</kbd> again. Repeat this process until the end of the notebook. <b>NOTE:</b> All the cells in this notebook can be automatically executed sequentially by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart and Run All</kbd>. Should anything crash then restart the Jupyter Kernal by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart</kbd>, and start again from the top.
        
</div>

<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
<img src="https://github.com/CIMCB/MetabComparisonBinaryML/blob/master/cimcb_logo.png?raw=true" width="180px" align="right" style="padding: 20px">

<h1> PLSDA_ST000496 </h1>

<br>
<br>
<br>
<p style="text-align: justify"> The study used in this tutorial has been previously published by  <a href="https://www.ncbi.nlm.nih.gov/pubmed/28220901">Sakanaka et al. (2017)</a>, and the deconvolved and annotated data file deposited at the Metabolomics Workbench data repository (Study ID: ST000496). The data can be accessed directly via its project DOI: <a href="http://dx.doi.org/DOI:10.21228/M8TW22">10.21228/M8TW22</a>. This workflow requires data to be formatted as a Microsoft Excel file, using the Tidy Data framework (i.e. each column is a variable, and row is an observation). As such, the Excel file contains a Data Sheet and Peak Sheet. The Data Sheet contains all the metabolite concentrations and metadata associated with each observation (requiring the inclusion of the columns: Idx, SampleID, and Class). The Peak Sheet contains all the metadata pertaining to each measured metabolite (requiring the inclusion of the columns: Idx, Name, and Label). Please inspect the Excel file <a href="https://github.com/CIMCB/MetabComparisonBinaryML/blob/master/dynamic/data/ST000496.xlsx?raw=true">ST000496.xlsx</a> used in this workflow before proceeding.</p>

<p style="text-align: justify">This is a saliva GC-MS dataset consisting of 69 named metabolites. The primary outcome for this paper was to assess the correlation of periodontal inflamed surface area (PISA) and salivary metabolites, before and after debridement. For the purpose of this publication, we compare the secondary outcome in a binary discriminant analysis: before debridement (Class=0; n=50) and after debridement (Class=1; n=50).</p>

<br>


</ol> 

</div>

<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
    
<h1> PLS-DA Workflow </h1> 
<br>
This Jupyter notebook implements the complete workflow for creating, optimising, and evaluating a partial least squares discriminant analysis (PLS-DA) model. <b style="text-align: justify"> PLS-DA was implemented using the SIMPLS algorithm. Refer to <a href="https://www.sciencedirect.com/science/article/abs/pii/016974399385002X">De Jong (1993)</a> for details on the SIMPLS algorithm.</b>
<br>
<i style="text-align: justify"> Please refer to the 'cimcb' package documentation for further details regarding this specific implementation: <a href="https://cimcb.github.io/cimcb">https://cimcb.github.io/cimcb</a></i>
 
<br>
<br>
<b style="text-align: justify"> PLS-DA uses the following Hyperparameter(s):</b>
<ul style="list-style-type: square;">
  <li><code>n_components</code>: number of latent variable (components) used in the PLS projection (default = 2)</li>
</ul>
<i style="text-align: justify">The purpose of each hyperparameter is explained here:  <a href="https://www.sciencedirect.com/science/article/abs/pii/016974399385002X">De Jong (1993)</a></i>
 
<br>
<br>

<b style="text-align: justify"> The notebook workflow is broken into the following steps:</b>

<ol>
    <li><b><i>Import Packages</i></b>: First, the Python packages required for this workflow need to be imported (<a href="http://www.numpy.org/"><code>numpy</code></a>, <a href="https://pandas.pydata.org/"><code>pandas</code></a>, and <a href="https://cimcb.github.io/cimcb"><code>cimcb</code></a>).
</li>
    <li><b><i>Load Data & Peak Sheet:</i></b> From the Excel spreadsheet, import the Data and Peak spreadsheets and create two respective <a href="https://pandas.pydata.org/">Pandas</a> tables: <code>DataTable</code> and <code>PeakTable</code>.</li>
    <li><b><i>Extract X & Y:</i></b> Next, we reduce the data in <code>DataTable</code> to include only those observations needed for the binary comparison and create a new table: <code>DataTable2</code>. We define one column of the data table to be the "outcome" variable <code>Outcomes</code>, and convert the class labels in this column to a binary outcome vector <code>Y</code>, where <code>1</code> is the positive outcome, and <code>0</code> the negative outcome (eg. case=1 & control=0). A new variable <code>peaklist</code> is created to hold the names (M1...Mn) of the metabolites to be used in the discriminant analysis. To create an independent dataset to evaluate, <a href="https://scikit-learn.org/stable/">scikit-learn</a> module's <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html"><code>train_test_split()</code></a> function is used. The data is split into 2/3rd training (<code>DataTrain</code> and <code>YTrain</code>), and 1/3rd test (<code>DataTest</code> and <code>YTest</code>). The metabolite data corresponding to <code>peaklist</code> is extracted from <code>DataTrain</code> and placed in a matrix <code>XTrain</code>. The <code>XTrain</code> matrix is log-transformed and auto-scaled, with missing values imputed using k-nearest neighbours (k=3). Then the metabolite data corresponding to <code>peaklist</code> is extracted from <code>DataTest</code> and placed in a matrix <code>XTest</code>. The <code>XTest</code> matrix is log-transformed and auto-scaled (using mu and sigma from <code>XTrain</code>), with missing values imputed using k-nearest neighbours (k=3).
    <li><b><i>Hyperparameter Optimisation:</i></b> Here, we use the helper function <code>cb.cross_val.kfold()</code> to carry out 5-fold cross-validation of a set of PLS-DA models configured with different numbers of latent variables (1 to 6). This helper function is generally applicable, and the values being passed to it are: 
    <ul>
    <li>The class of model to be created by the function, <code>cb.model.PLS_SIMPLS</code>.</li>
        <li>The metabolite matrix, <code>XTknn</code>, and binary outcome vector, <code>Y</code>.</li>
        <li>A dictionary, <code>param_dict</code>, describing key:value pairs where the key is a parameter that is passed to the model, and the value is a list of values to be passed to that parameter.</li>
        <li>The number of folds in the cross-validation, <code>folds</code>, and the number of Monte Carlo repetitions of the k-fold CV, <code>n_mc</code>.</li></ul>
When <code>cv.run()</code> followed by <code>cv.plot(metric='r2q2')</code> are run the results are displayed as 2 plots of $R^2$ and $Q^2$ statistics: (a) the difference ($R^2 - Q^2$) vs. $Q^2$, and (b) absolute values of both $R^2$ and $Q^2$ against the number of components. These plots are used to aid in selecting the optimal number of components.</li>
    <li><b><i>Build Model & Evaluate:</i></b> Here, we use the class <code>cb.model.PLS_SIMPLS()</code> to building a PLS-DA model using the optimal hyperparameter values determined in step 4. The model is trained on the training dataset, <code>XTrainKnn</code>, and tested on the independent test dataset, <code>XTestKnn</code>. Next, the trained model's <code>.evaluate()</code> method is used to visualise model performance for both the training and independent test dataset using: a <a href="https://www.data-to-viz.com/graph/violin.html">violin plot</a> showing the distributions of negative and positive responses as violin and box-whisker plots; a <a href="https://books.google.com.au/books?id=7WBMrZ9umRYC">probability density function</a> plot for each response type, and a <a href="https://doi.org/10.1007/s11306-012-0482-9">ROC curve</a> for the classifier, with 95% confidence interval (lighter shaded area). 
   <li><b><i>Bootstrap Evaluation:</i></b> Finally, to create an estimate of the robustness and a measure of generalised predictive ability of this model we perform  <a href="https://link.springer.com/article/10.1007%2FBF00058655">bootstrap aggregation</a> (Bagging) using the model's <code>.booteval()</code> method (100 bootstrapped models). This generates a population of 100 model predictions for both the training set (in-bag prediction - IB) and the holdout test set (out-of-bag - OOB). These predictions are visualised with a box-violin and probability density function plot for the aggregate model. The ROC curve displays the curve for the aggregate model (green line = IB & red line = OOB) with 95% confidence interval for summarising the predictions for the 100 constituent models, using the <a href="https://www.tandfonline.com/doi/abs/10.1080/01621459.1987.10478410">bias-corrected and accelerated method</a>. Additionally, the ROC curve is overlayed with the curve from the independent test dataset.
  <li><b><i>Export Results:</i></b> Exporting the model evaluation results as an Excel spreadsheet.</li>
</ol> 

</div>

<div style="background-color:rgb(240,248,255); border: 1px solid lightgrey; padding:5px; padding-left: 1em; padding-right: 1em;">
    
<a id="1"></a>
<h3 style="text-align: justify"><i>Set Random Seed</i></h3>

<p style="text-align: justify">To simulate an independent test set, the data is split into 1/3rd training and 2/3rd test using <a href="https://scikit-learn.org/stable/">scikit-learn</a> module's <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html"><code>train_test_split()</code></a> function. To ensure this independent test set is consistent across all workflows, a random seed is used. For all workflows, the seed is set to 44. 

<br>
 
</div>

In [1]:

seed = 44

### 1. Import Packages

In [2]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

Using Theano backend.


All packages successfully loaded


### 2. Load Data & Peak Sheet

In [3]:
home = 'data/' 
file = 'ST000496.xlsx'  

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 100 TOTAL PEAKS: 69
Done!


### 3. Extract X & Y

In [4]:
# Select Subset of Data
DataTable2 = DataTable[(DataTable.Class == 1) | (DataTable.Class == 0)]

# Create a Binary Y Vector 
Outcomes = DataTable2['Class']
Y = Outcomes.values 

# Split Data into Train (2/3) and Test (1/3)
DataTrain, DataTest, YTrain, YTest = train_test_split(DataTable2, Y, test_size=1/3, stratify=Y, random_state=seed)

# Extract Train Data 
peaklist = PeakTable['Name']                           
XTrain = DataTrain[peaklist]                                    
XTrainLog = np.log(XTrain)                                          
XTrainScale, mu, sigma = cb.utils.scale(XTrainLog, method='auto', return_mu_sigma=True)              
XTrainKnn = cb.utils.knnimpute(XTrainScale, k=3)    

# Extract Test Data
peaklist = PeakTable['Name']                           
XTest = DataTest[peaklist]                                    
XTestLog = np.log(XTest)                                          
XTestScale = cb.utils.scale(XTestLog, method='auto', mu=mu, sigma=sigma) # Use Test Data Scaling            
XTestKnn = cb.utils.knnimpute(XTestScale, k=3)                                                                     

### 4. Hyperparameter Optimisation

In [5]:
# Parameter Dictionary
param_dict = {'n_components': [1, 2, 3, 4, 5, 6]}                   

# Initialise
cv = cb.cross_val.KFold(model=cb.model.PLS_SIMPLS,                      
                        X=XTrainKnn,                                 
                        Y=YTrain,                               
                        param_dict=param_dict,                   
                        folds=5,
                        n_mc=100)       

# Run and Plot
cv.run()  
cv.plot(metric='r2q2', method='ratio', ci=95) 

Number of cores set to: 8
Running ...


2/2: 100%|██████████| 600/600 [00:02<00:00, 233.55it/s]


Time taken: 0.21 minutes with 8 cores
Done!


Loading BokehJS ...

### 5. Build Model & Evaluate

In [7]:
# Build Model
model = cb.model.PLS_SIMPLS(n_components=3)
YPredTrain = model.train(XTrainKnn, YTrain)
YPredTest = model.test(XTestKnn)

# Put YTest and YPrestTest in a list
EvalTest = [YTest, YPredTest]

# Evaluate Model
model.evaluate(testset=EvalTest, bootnum=100, bootmethod='Per') # Per, CPer, BCA 

Loading BokehJS ...

### 6. Bootstrap Evaluation

In [8]:
# Build Boostrap Models
bootmodel = cb.bootstrap.Per(model, bootnum=100) # Per, CPer, BCA
bootmodel.run()

# Boostrap Evaluate Model (include Test Dataset)
bootmodel.evaluate(testset=EvalTest, plot_roc='median')   #plot_roc='data'?

Number of cores set to: 8


100%|██████████| 100/100 [00:00<00:00, 333.87it/s]


Time taken: 0.01 minutes with 8 cores


Loading BokehJS ...

### 7. Save Table

In [9]:
home = 'results/'
file = 'PLSDA_ST000496.xlsx'

bootmodel.save_results(home + file)

Done! Saved results as results/PLSDA_ST000496.xlsx
